# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 29 2022 8:30AM,253960,10,0086271557,ISDIN Corporation,Released
1,Dec 29 2022 8:30AM,253964,10,0086271559,ISDIN Corporation,Released
2,Dec 29 2022 8:30AM,253964,10,0086271560,ISDIN Corporation,Released
3,Dec 29 2022 8:30AM,253964,10,0086271558,ISDIN Corporation,Released
4,Dec 29 2022 8:30AM,253964,10,0086271562,ISDIN Corporation,Released
5,Dec 29 2022 8:30AM,253953,10,Eme-106947,Emerginnova,Released
6,Dec 29 2022 8:30AM,253957,10,8960783,"Carwin Pharmaceutical Associates, LLC",Released
7,Dec 28 2022 4:24PM,253963,17,8960788-D,"ACI Healthcare USA, Inc.",Released
8,Dec 28 2022 4:12PM,253961,17,8960786-T-T.1,"Brookfield Pharmaceuticals, LLC",Released
9,Dec 28 2022 4:12PM,253961,17,8960786-T,"Brookfield Pharmaceuticals, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,253958,Released,1
42,253960,Released,1
43,253961,Released,2
44,253963,Released,1
45,253964,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253958,NaN,1.0
253960,NaN,1.0
253961,NaN,2.0
253963,NaN,1.0
253964,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253863,0.0,1.0
253865,0.0,38.0
253868,0.0,3.0
253870,0.0,7.0
253876,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253863,0,1
253865,0,38
253868,0,3
253870,0,7
253876,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253863,0,1
1,253865,0,38
2,253868,0,3
3,253870,0,7
4,253876,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253863,,1
1,253865,,38
2,253868,,3
3,253870,,7
4,253876,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 29 2022 8:30AM,253960,10,ISDIN Corporation
1,Dec 29 2022 8:30AM,253964,10,ISDIN Corporation
5,Dec 29 2022 8:30AM,253953,10,Emerginnova
6,Dec 29 2022 8:30AM,253957,10,"Carwin Pharmaceutical Associates, LLC"
7,Dec 28 2022 4:24PM,253963,17,"ACI Healthcare USA, Inc."
8,Dec 28 2022 4:12PM,253961,17,"Brookfield Pharmaceuticals, LLC"
10,Dec 28 2022 4:00PM,253958,17,"Carwin Pharmaceutical Associates, LLC"
11,Dec 28 2022 3:55PM,253954,15,"Alliance Pharma, Inc."
13,Dec 28 2022 3:37PM,253952,19,ACG North America LLC
14,Dec 28 2022 3:05PM,253949,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 29 2022 8:30AM,253960,10,ISDIN Corporation,,1
1,Dec 29 2022 8:30AM,253964,10,ISDIN Corporation,,4
2,Dec 29 2022 8:30AM,253953,10,Emerginnova,,1
3,Dec 29 2022 8:30AM,253957,10,"Carwin Pharmaceutical Associates, LLC",,1
4,Dec 28 2022 4:24PM,253963,17,"ACI Healthcare USA, Inc.",,1
5,Dec 28 2022 4:12PM,253961,17,"Brookfield Pharmaceuticals, LLC",,2
6,Dec 28 2022 4:00PM,253958,17,"Carwin Pharmaceutical Associates, LLC",,1
7,Dec 28 2022 3:55PM,253954,15,"Alliance Pharma, Inc.",2,
8,Dec 28 2022 3:37PM,253952,19,ACG North America LLC,,1
9,Dec 28 2022 3:05PM,253949,19,"GUSA Granules USA, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 8:30AM,253960,10,ISDIN Corporation,1,
1,Dec 29 2022 8:30AM,253964,10,ISDIN Corporation,4,
2,Dec 29 2022 8:30AM,253953,10,Emerginnova,1,
3,Dec 29 2022 8:30AM,253957,10,"Carwin Pharmaceutical Associates, LLC",1,
4,Dec 28 2022 4:24PM,253963,17,"ACI Healthcare USA, Inc.",1,
5,Dec 28 2022 4:12PM,253961,17,"Brookfield Pharmaceuticals, LLC",2,
6,Dec 28 2022 4:00PM,253958,17,"Carwin Pharmaceutical Associates, LLC",1,
7,Dec 28 2022 3:55PM,253954,15,"Alliance Pharma, Inc.",,2
8,Dec 28 2022 3:37PM,253952,19,ACG North America LLC,1,
9,Dec 28 2022 3:05PM,253949,19,"GUSA Granules USA, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 8:30AM,253960,10,ISDIN Corporation,1,
1,Dec 29 2022 8:30AM,253964,10,ISDIN Corporation,4,
2,Dec 29 2022 8:30AM,253953,10,Emerginnova,1,
3,Dec 29 2022 8:30AM,253957,10,"Carwin Pharmaceutical Associates, LLC",1,
4,Dec 28 2022 4:24PM,253963,17,"ACI Healthcare USA, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 8:30AM,253960,10,ISDIN Corporation,1.0,NaN
1,Dec 29 2022 8:30AM,253964,10,ISDIN Corporation,4.0,NaN
2,Dec 29 2022 8:30AM,253953,10,Emerginnova,1.0,NaN
3,Dec 29 2022 8:30AM,253957,10,"Carwin Pharmaceutical Associates, LLC",1.0,NaN
4,Dec 28 2022 4:24PM,253963,17,"ACI Healthcare USA, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 29 2022 8:30AM,253960,10,ISDIN Corporation,1.0,0.0
1,Dec 29 2022 8:30AM,253964,10,ISDIN Corporation,4.0,0.0
2,Dec 29 2022 8:30AM,253953,10,Emerginnova,1.0,0.0
3,Dec 29 2022 8:30AM,253957,10,"Carwin Pharmaceutical Associates, LLC",1.0,0.0
4,Dec 28 2022 4:24PM,253963,17,"ACI Healthcare USA, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,9140959,306.0,0.0
15,253954,0.0,2.0
17,761882,4.0,0.0
19,1523531,28.0,0.0
21,253916,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,9140959,306.0,0.0
1,15,253954,0.0,2.0
2,17,761882,4.0,0.0
3,19,1523531,28.0,0.0
4,21,253916,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,306.0,0.0
1,15,0.0,2.0
2,17,4.0,0.0
3,19,28.0,0.0
4,21,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,306.0
1,15,Released,0.0
2,17,Released,4.0
3,19,Released,28.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,17,19,21
Status,,,,,
Executing,0.0,2.0,0.0,0.0,0.0
Released,306.0,0.0,4.0,28.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,17,19,21
0,Executing,0.0,2.0,0.0,0.0,0.0
1,Released,306.0,0.0,4.0,28.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,17,19,21
0,Executing,0.0,2.0,0.0,0.0,0.0
1,Released,306.0,0.0,4.0,28.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()